#시간 상 부족해서 직접 구현은 포기

In [1]:
import numpy as np

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))    
def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)

In [4]:
class Perceptron:
    def __init__(self, input_size, learning_rate=0.001, epochs=100):
        self.weights = np.zeros(input_size + 1)  # +1 for bias
        self.learning_rate = learning_rate
        self.epochs = epochs
        
    def activation_function(self, x):
        return sigmoid(x)
        
    def fit(self, X, y):
        X = np.c_[np.ones(X.shape[0]), X]

        for epoch in range(self.epochs):
            for i in range(X.shape[0]):
                linear_output = np.dot(X[i], self.weights)
                y_pred = self.activation_function(linear_output)
                self.weights += self.learning_rate * sigmoid_grad((y[i] - y_pred) * X[i])
    def predict(self, X):
        X = np.c_[np.ones(X.shape[0]), X]
        y_pred = [self.activation_function(np.dot(x, self.weights)) for x in X]
        return np.array(y_pred)        


In [14]:
class Convolution:
    def __init__(self, num_filters, filter_size, input_shape, stride=1, padding=0):
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.stride = stride
        self.padding = padding
        self.input_shape = input_shape
        self.filters = np.random.randn(num_filters, input_shape[0], filter_size, filter_size) / np.sqrt(filter_size * filter_size)

    def pad_input(self, X):
        if self.padding == 0:
            return X
        return np.pad(X, ((0, 0), (0, 0), (self.padding, self.padding), (self.padding, self.padding)), mode='constant')

    def convolve(self, x, filt):
        c, h, w = filt.shape
        out_h = (x.shape[1] - h) // self.stride + 1
        out_w = (x.shape[2] - w) // self.stride + 1
        out = np.zeros((out_h, out_w))
        for i in range(0, out_h):
            for j in range(0, out_w):
                region = x[:, i * self.stride:i * self.stride + h, j * self.stride:j * self.stride + w]
                out[i, j] = np.sum(region * filt)
        return out

    def forward(self, X):
        X_padded = self.pad_input(X)
        batch_size, _, _, _ = X_padded.shape
        output_height = (X_padded.shape[2] - self.filter_size) // self.stride + 1
        output_width = (X_padded.shape[3] - self.filter_size) // self.stride + 1
        output = np.zeros((batch_size, self.num_filters, output_height, output_width))
        for i in range(batch_size):
            for j in range(self.num_filters):
                output[i, j] = self.convolve(X_padded[i], self.filters[j])
        return output